In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model

2025-05-30 00:13:08.694241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = load_model('model.h5')

2025-05-30 00:13:18.313783: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:


model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input.1 (InputLayer)        [(None, 500, 6, 1)]       0         
                                                                 
 conv_Conv_output_0 (Conv2D)  (None, 500, 1, 64)       448       
                                                                 
 temp_conv_temp_conv.0_Conv_  (None, 502, 1, 64)       0         
 output_0_pad (ZeroPadding2D                                     
 )                                                               
                                                                 
 temp_conv_temp_conv.0_Conv_  (None, 250, 1, 32)       6176      
 output_0 (Conv2D)                                               
                                                                 
 temp_conv_temp_conv.1_Conv_  (None, 252, 1, 32)       0         
 output_0_pad (ZeroPadding2D                                 

In [13]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Permute, Flatten
import tensorflow as tf

lambda_layer = model.get_layer('Flatten_output_0_CHW')
prev_layer = lambda_layer.input

# Step 2: Insert a Permute layer instead of Lambda
# Assuming original output was (None, 64, 125, 1) and we want to go to (None, 125, 1, 64)


# Step 3: Continue the rest of the model
# Get next layers after Lambda
flatten = model.get_layer('Flatten_output_0')(prev_layer)
fc1 = model.get_layer('fc_fc.0_Gemm_output_0')(flatten)
act = model.get_layer('fc_fc.1_Relu_output_0')(fc1)
out = model.get_layer('25')(act)

# Step 4: Build new model
patched_model = Model(inputs=model.input, outputs=out)

# Optional: compile (only needed if training or using metrics/loss)
patched_model.compile(optimizer='adam', loss='categorical_crossentropy')

# Save the fixed model
patched_model.save('model_no_lambda.h5')